## Mana-fest Destiny

In [2]:
#Imports
import os
import pandas as pd
import requests

### Load/Download and Preclean Data

In [8]:
# Set file paths
DATA_DIR = "data/"
DRAFT_CSV = os.path.join(DATA_DIR, "cleaned_data.csv")
SCRYFALL_CSV = os.path.join(DATA_DIR, "bloomburrow_cleaned.csv")

# Ensure data directory exists
os.makedirs(DATA_DIR, exist_ok=True)

# Function to load draft data (skip download if it exists)
def load_draft_data():
    if os.path.exists(DRAFT_CSV):
        print("Loading draft data from existing file...")
        return pd.read_csv(DRAFT_CSV)
    else:
        print("Draft file missing. ERROR. CANNOT PROCEED. SELF DESTRUCT IN 5, 4, 3, 2, 1")
        assert(False)

# Function to fetch Scryfall data (skip if already saved)
def fetch_scryfall_data():
    if os.path.exists(SCRYFALL_CSV):
        print("Loading Scryfall data from existing file...")
        return pd.read_csv(SCRYFALL_CSV)
    else:
        print("Fetching Scryfall API data...")
        url = "https://api.scryfall.com/cards/search?q=set:blb"
        response = requests.get(url)
        data = response.json()["data"]
        df = pd.DataFrame(data)
        df.to_csv(SCRYFALL_CSV, index=False)
        return df

# Load datasets with idempotency
draft_df = load_draft_data()
card_df = fetch_scryfall_data()

Loading draft data from existing file...
Loading Scryfall data from existing file...


In [5]:
def clean_card_data(card_df: pd.DataFrame, output_file: str) -> None:
    """Cleans the card dataset by dropping unnecessary columns and saving the processed version."""
    core_card_data = ["name", "mana_cost", "cmc", "type_line", "oracle_text", "colors", "color_identity", "keywords", "rarity", "power", "toughness"]
    skeptical_keepers = ["reprint"]
    external_references = ["oracle_id", "multiverse_ids", "mtgo_id", "arena_id", "tcgplayer_id", "cardmarket_id"]
    status_and_printing = ["foil", "nonfoil", "promo", "reprint", "variation", "security_stamp", "frame", "full_art", "textless"]
    art_and_flavor = ["artist", "illustration_id", "flavor_text", "border_color"]
    marketplace_and_pricing = ["prices", "purchase_uris", "related_uris"]
    metadata_and_links = ["set", "set_name", "set_type", "set_uri", "set_search_uri", "scryfall_set_uri", "rulings_uri", "prints_search_uri", "collector_number"]

    drop_list = external_references + status_and_printing + art_and_flavor + marketplace_and_pricing + metadata_and_links
    
    card_df.drop(columns=drop_list, inplace=True)
    card_df.to_csv(output_file, index=False)
    print(f"Processed data: {card_df.shape[0]} rows, {card_df.shape[1]} columns")
    print(f"Cleaned file saved as {output_file}")

# Example usage
raw_file = "bloomburrow_raw.csv"
clean_file = "bloomburrow_cleaned.csv"

clean_card_data(card_df, clean_file)

Processed data: 175 rows, 40 columns
Cleaned file saved as bloomburrow_cleaned.csv


### Combine Card/Deck Data, Basic Count Analysis

In [10]:
draft_df.columns

Index(['draft_id', 'draft_ti fome', 'game_time', 'build_index', 'match_number',
       'game_number', 'rank', 'opp_rank', 'main_colors', 'splash_colors',
       ...
       'tutored_Ygra, Eater of All', 'deck_Ygra, Eater of All',
       'sideboard_Ygra, Eater of All', 'opening_hand_Zoraline, Cosmos Caller',
       'drawn_Zoraline, Cosmos Caller', 'tutored_Zoraline, Cosmos Caller',
       'deck_Zoraline, Cosmos Caller', 'sideboard_Zoraline, Cosmos Caller',
       'user_n_games_bucket', 'user_game_win_rate_bucket'],
      dtype='object', length=1398)

In [14]:
# Step 4: Extract Card Types
def get_card_types(card_df: pd.DataFrame) -> set:
    """Finds all distinct card types in the dataset."""
    return {type_line.split()[0] for type_line in card_df["type_line"]}

# Step 5: Compute Deck Data
def generate_deck_data(draft_df: pd.DataFrame, card_df: pd.DataFrame, output_file: str, max_decks: int = None) -> None:
    """Aggregates deck performance data with an optional limit and saves it to a CSV file."""
    card_types = get_card_types(card_df)
    #deck_columns = ["deck_id", "wins", "losses", "avg_mana_curve", "bomb_density", "color_identity"] + [f"num_{ctype.lower()}" for ctype in card_types]
    
    # Draft id from that DF just becomes "deck_id" in the final
    grouped = draft_df.groupby("draft_id")
    deck_columns = [col for col in draft_df.columns if col.startswith("deck_")]
    deck_df = pd.DataFrame(columns=deck_columns)

    for i, (deck_id, group) in enumerate(grouped):
        if max_decks and i >= max_decks:
            break  # Stop early if max_decks is reached
        
        # Get list of card names from relevant columns
        deck_list = [col.replace("deck_", "") for col in deck_columns if group[col].sum() > 0]

        wins, losses = group["wins"].sum(), group["losses"].sum()

        non_land_cards = [card for card in deck_list if "Land" not in card_df.loc[card, "type_line"]]
        avg_mana_curve = sum(card_df.loc[card, "cmc"] for card in non_land_cards if card in card_df.index) / len(non_land_cards) if non_land_cards else 0
        bomb_density = sum(1 for card in deck_list if card_df.loc[card, "rarity"] in ["rare", "mythic"]) / len(deck_list)
        color_identity = list(set(color for card in deck_list for color in card_df.loc[card, "color_identity"]))
        type_counts = {f"num_{ctype.lower()}": sum(1 for card in deck_list if ctype in card_df.loc[card, "type_line"]) for ctype in card_types}

        deck_df.loc[len(deck_df)] = {**{"deck_id": deck_id, "wins": wins, "losses": losses, "avg_mana_curve": avg_mana_curve, "bomb_density": bomb_density, "color_identity": color_identity}, **type_counts}
    deck_df.to_csv(output_file, index=False)
    print(f"Deck DataFrame created: {deck_df.shape[0]} rows, {deck_df.shape[1]} columns (Processed up to {max_decks} decks)")


generate_deck_data(draft_df, card_df, "first_analysis.csv", 2)


KeyError: 'wins'

In [ ]:
def analyze_first_full_draft(file_path="game_data_public.BLB.PremierDraft.csv"):
    """
    Reads game-level data, finds the first complete draft, calculates its win rate,
    and displays its decklist.
    """

    print(f"Attempting to read data from: {file_path}")

    # Strategy: Read in chunks or a larger initial number of rows
    # to ensure we capture at least one full draft, then trim.
    # A typical draft is 3-9 games, so 50 rows should be more than enough.
    chunk_size = 50
    df = pd.read_csv(file_path, nrows=chunk_size)

    # Identify the first draft_id
    first_draft_id = df['draft_id'].iloc[0]
    print(f"\nAnalyzing data for the first draft_id: {first_draft_id}")

    # Filter to get all rows belonging to this first draft_id
    first_draft_df = df[df['draft_id'] == first_draft_id].copy()

    if first_draft_df.empty:
        print(f"Error: Could not find full data for draft_id {first_draft_id} within the first {chunk_size} rows. Try increasing chunk_size.")
        return

    # --- Calculate Win Rate ---
    # We need to sum the 'won' column for the first_draft_df to get total wins
    # and count total games to get total losses
    total_wins = first_draft_df['won'].sum()
    total_games = len(first_draft_df) # Each row is a game
    total_losses = total_games - total_wins

    if total_games > 0:
        win_rate = (total_wins / total_games) * 100
        print(f"\n--- Draft Performance for {first_draft_id} ---")
        print(f"Event record: {total_wins}-{total_losses}")
        print(f"Win Rate: {win_rate:.2f}%")
    else:
        print(f"\n--- Draft Performance for {first_draft_id} ---")
        print("No games played in this event (total_games = 0).")

    # --- Extract Decklist ---
    # Find all columns that start with 'deck_'
    deck_card_columns = [col for col in first_draft_df.columns if col.startswith('deck_')]

    # Assuming the decklist is constant for all games within the draft,
    # we can just take the first row's deck data.
    # We want card names where the count is > 0.
    deck_composition_row = first_draft_df[deck_card_columns].iloc[0]

    # Create a dictionary of card_name: count
    decklist_raw = {
        col.replace('deck_', ''): int(deck_composition_row[col])
        for col in deck_composition_row.index
        if deck_composition_row[col] > 0
    }

    # Format the decklist for display
    print("\n--- Decklist ---")
    if not decklist_raw:
        print("No cards found in the deck (or 'deck_' columns not present/empty).")
    else:
        # Separate lands from spells for better readability if possible
        # (This would be more robust with the Scryfall data, but we can try a heuristic)
        lands = {}
        spells = {}

        for card_name, count in decklist_raw.items():
            # Simple heuristic: if it contains 'Forest', 'Island', etc., or is low cmc
            # This is NOT robust and would be better with Scryfall's 'type_line'
            if 'Forest' in card_name or 'Island' in card_name or 'Swamp' in card_name or \
               'Mountain' in card_name or 'Plains' in card_name or 'Wastes' in card_name:
                lands[card_name] = count
            else:
                spells[card_name] = count

        if lands:
            print("Lands:")
            for card, count in sorted(lands.items()):
                print(f"  {count}x {card}")
        if spells:
            print("Spells:")
            for card, count in sorted(spells.items()):
                print(f"  {count}x {card}")

    print("\n--- Raw Decklist (Card: Count) ---")
    print(decklist_raw)


# --- Run the analysis ---
if __name__ == "__main__":
    analyze_first_full_draft()

Attempting to read data from: game_data_public.BLB.PremierDraft.csv

Analyzing data for the first draft_id: deaa4cdcd3e84d8e8b5a0ea34a0f9d79

--- Draft Performance for deaa4cdcd3e84d8e8b5a0ea34a0f9d79 ---
Event record: 6-3
Win Rate: 66.67%

--- Decklist ---
Lands:
  9x Forest
  8x Swamp
Spells:
  1x Bakersbane Duo
  2x Cache Grab
  1x Camellia, the Seedmiser
  1x Cindering Cutthroat
  1x Daggerfang Duo
  1x Downwind Ambusher
  1x Druid of the Spade
  1x Glidedive Duo
  1x Hazardroot Herbalist
  1x Hazel's Nocturne
  1x Head of the Homestead
  1x Longstalk Brawl
  1x Overprotect
  1x Polliwallop
  2x Savor
  1x Tangle Tumbler
  1x Tender Wildguide
  1x Thought-Stalker Warlock
  2x Three Tree Rootweaver
  1x Ygra, Eater of All

--- Raw Decklist (Card: Count) ---
{'Bakersbane Duo': 1, 'Cache Grab': 2, 'Camellia, the Seedmiser': 1, 'Cindering Cutthroat': 1, 'Daggerfang Duo': 1, 'Downwind Ambusher': 1, 'Druid of the Spade': 1, 'Forest': 9, 'Glidedive Duo': 1, 'Hazardroot Herbalist': 1, "Haz

In [ ]:
import pandas as pd

# Load datasets
draft_df = pd.read_csv("cleaned_data.csv")
card_df = pd.read_csv("bloomburrow_cleaned.csv").set_index("name")

# Find all distinct card types in Bloomburrow
card_types = set()
for type_line in card_df["type_line"]:
    main_type = type_line.split()[0]  # Take the first word (e.g., "Creature", "Artifact")
    card_types.add(main_type)

# Create empty Deck DataFrame with dynamic type columns
deck_columns = ["deck_id", "wins", "losses", "avg_mana_curve", "bomb_density", "color_identity"] + [f"num_{ctype.lower()}" for ctype in card_types]
deck_df = pd.DataFrame(columns=deck_columns)

# Aggregate deck data
grouped = draft_df.groupby("deck_id")

for deck_id, group in grouped:
    deck_list = group["card_name"].tolist()

    # Compute core metrics
    wins = group["wins"].sum()
    losses = group["losses"].sum()
    
    # Filter out lands before calculating mana curve
    non_land_cards = [card for card in deck_list if "Land" not in card_df.loc[card, "type_line"]]
    avg_mana_curve = sum(card_df.loc[card, "cmc"] for card in non_land_cards if card in card_df.index) / len(non_land_cards) if non_land_cards else 0

    bomb_density = sum(1 for card in deck_list if card_df.loc[card, "rarity"] in ["rare", "mythic"]) / len(deck_list)
    color_identity = list(set(color for card in deck_list for color in card_df.loc[card, "color_identity"]))

    # Count card types dynamically
    type_counts = {f"num_{ctype.lower()}": sum(1 for card in deck_list if ctype in card_df.loc[card, "type_line"]) for ctype in card_types}

    # Append to deck_df
    deck_df.loc[len(deck_df)] = {
        **{"deck_id": deck_id, "wins": wins, "losses": losses, "avg_mana_curve": avg_mana_curve, "bomb_density": bomb_density, "color_identity": color_identity},
        **type_counts
    }

# Save the structured deck data
deck_df.to_csv("deck_analysis.csv", index=False)

print(f"Deck DataFrame created: {deck_df.shape[0]} rows, {deck_df.shape[1]} columns")


### 5/30 parquet and other work


In [3]:
import pandas as pd

# Load your dataset (assuming it's a CSV)
csv_file = "data/bloomburrow/games.csv"
df = pd.read_csv(csv_file)

# Save as Parquet (PyArrow format)
parquet_file = "data/bloomburrow/games.parquet"
df.to_parquet(parquet_file, engine="pyarrow", compression="snappy")  # Snappy is fast & efficient

print(f"Converted {csv_file} to {parquet_file}.")


Converted data/bloomburrow/games.csv to data/bloomburrow/games.parquet.


In [2]:
df.head(2)

,object,id,oracle_id,multiverse_ids,mtgo_id,arena_id,tcgplayer_id,cardmarket_id,name,lang,...,prices,related_uris,purchase_uris,power,toughness,all_parts,security_stamp,promo_types,frame_effects,produced_mana
0,card,7dd9946b-515e-4e0d-9da2-711e126e9fa6,7b89b7d2-c724-4d5d-9f0b-7d3302ad1168,[669036],129489.0,91658.0,559491.0,778435.0,Agate Assault,en,...,"{'usd': '0.03', 'usd_foil': '0.06', 'usd_etche...",{'gatherer': 'https://gatherer.wizards.com/Pag...,{'tcgplayer': 'https://partner.tcgplayer.com/c...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,card,39ebb84a-1c52-4b07-9bd0-b360523b3a5b,381a3e8e-71dd-48e4-ab62-53478bde4a14,[668996],129409.0,91618.0,559647.0,778511.0,Agate-Blade Assassin,en,...,"{'usd': '0.04', 'usd_foil': '0.07', 'usd_etche...",{'gatherer': 'https://gatherer.wizards.com/Pag...,{'tcgplayer': 'https://partner.tcgplayer.com/c...,1,3,NaN,NaN,NaN,NaN,NaN


In [8]:
import pandas as pd

# Load the Parquet file we just created
parquet_filestr = "data/bloomburrow/games.parquet"
# if 'games_df' not in locals() or games_df.empty:
#     games_df = pd.read_parquet(parquet_filestr, engine="pyarrow")
if "games_df" not in globals():
    games_df = pd.read_parquet(parquet_filestr, engine="pyarrow")
cards_df = pd.read_csv("data/bloomburrow/cards.csv")


In [ ]:
# Demonstrate columnar efficiency by reading only a specific column

parquet_file = "data/bloomburrow/games.parquet"
df = pd.read_parquet(parquet_file, columns=["draft_id"], engine="pyarrow")

# Quick check: Print a few rows
print(df.head())

# Verify column type and memory usage
print(df.info())

                           draft_id
0  deaa4cdcd3e84d8e8b5a0ea34a0f9d79
1  deaa4cdcd3e84d8e8b5a0ea34a0f9d79
2  deaa4cdcd3e84d8e8b5a0ea34a0f9d79
3  deaa4cdcd3e84d8e8b5a0ea34a0f9d79
4  deaa4cdcd3e84d8e8b5a0ea34a0f9d79
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 931230 entries, 0 to 931229
Data columns (total 1 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   draft_id  931230 non-null  object
dtypes: object(1)
memory usage: 7.1+ MB
None


In [11]:
cards_df.columns

Index(['object', 'id', 'oracle_id', 'multiverse_ids', 'mtgo_id', 'arena_id',
       'tcgplayer_id', 'cardmarket_id', 'name', 'lang', 'released_at', 'uri',
       'scryfall_uri', 'layout', 'highres_image', 'image_status', 'image_uris',
       'mana_cost', 'cmc', 'type_line', 'oracle_text', 'colors',
       'color_identity', 'keywords', 'legalities', 'games', 'reserved',
       'game_changer', 'foil', 'nonfoil', 'finishes', 'oversized', 'promo',
       'reprint', 'variation', 'set_id', 'set', 'set_name', 'set_type',
       'set_uri', 'set_search_uri', 'scryfall_set_uri', 'rulings_uri',
       'prints_search_uri', 'collector_number', 'digital', 'rarity',
       'flavor_text', 'card_back_id', 'artist', 'artist_ids',
       'illustration_id', 'border_color', 'frame', 'full_art', 'textless',
       'booster', 'story_spotlight', 'edhrec_rank', 'penny_rank', 'preview',
       'prices', 'related_uris', 'purchase_uris', 'power', 'toughness',
       'all_parts', 'security_stamp', 'promo_types', '

In [10]:
def analyze_first_full_draft(games_df):
    """
    Takes a Pandas DataFrame (`games_df`), finds the first complete draft, 
    calculates its win rate, and returns the raw decklist.

    Returns:
        dict: Raw decklist with card names as keys and counts as values.
    """

    if games_df.empty:
        print("Error: DataFrame is empty. Cannot analyze draft.")
        return {}

    # Identify the first draft_id
    first_draft_id = games_df["draft_id"].iloc[0]
    print(f"\nAnalyzing data for draft_id: {first_draft_id}")

    # Filter for all rows belonging to this draft_id
    first_draft_df = games_df[games_df["draft_id"] == first_draft_id].copy()

    # --- Calculate Win Rate ---
    total_wins = first_draft_df["won"].sum()
    total_games = len(first_draft_df)
    total_losses = total_games - total_wins

    win_rate = (total_wins / total_games) * 100 if total_games > 0 else 0.0

    print(f"\n--- Draft Performance for {first_draft_id} ---")
    print(f"Event Record: {total_wins}-{total_losses}")
    print(f"Win Rate: {win_rate:.2f}%")

    # --- Extract Decklist ---
    deck_card_columns = [col for col in first_draft_df.columns if col.startswith("deck_")]
    deck_composition_row = first_draft_df[deck_card_columns].iloc[0]

    # Convert to dictionary format: {card_name: count}
    decklist_raw = {
        col.replace("deck_", ""): int(deck_composition_row[col])
        for col in deck_composition_row.index
        if deck_composition_row[col] > 0
    }

    return decklist_raw  # Now returning the decklist for further enrichment

decklist_raw = analyze_first_full_draft(games_df)
print("\n--- Raw Decklist ---")
print(decklist_raw)


Analyzing data for draft_id: deaa4cdcd3e84d8e8b5a0ea34a0f9d79

--- Draft Performance for deaa4cdcd3e84d8e8b5a0ea34a0f9d79 ---
Event Record: 6-3
Win Rate: 66.67%

--- Raw Decklist ---
{'Bakersbane Duo': 1, 'Cache Grab': 2, 'Camellia, the Seedmiser': 1, 'Cindering Cutthroat': 1, 'Daggerfang Duo': 1, 'Downwind Ambusher': 1, 'Druid of the Spade': 1, 'Forest': 9, 'Glidedive Duo': 1, 'Hazardroot Herbalist': 1, "Hazel's Nocturne": 1, 'Head of the Homestead': 1, 'Longstalk Brawl': 1, 'Overprotect': 1, 'Polliwallop': 1, 'Savor': 2, 'Swamp': 8, 'Tangle Tumbler': 1, 'Tender Wildguide': 1, 'Thought-Stalker Warlock': 1, 'Three Tree Rootweaver': 2, 'Ygra, Eater of All': 1}


In [18]:
# Convert decklist dictionary to DataFrame
decklist_df = pd.DataFrame(decklist_raw.items(), columns=["name", "count"])

# Merge with Scryfall card data
decklist_enriched = decklist_df.merge(cards_df[["name", "cmc", "type_line"]], on="name", how="left")

# Compute basic deck stats
avg_mana_value = (decklist_enriched["cmc"] * decklist_enriched["count"]).sum() / decklist_enriched["count"].sum()
num_creatures = decklist_enriched[decklist_enriched["type_line"].str.contains("Creature", na=False)]["count"].sum()

# Display enriched decklist with stats
print(f"\n--- Enhanced Deck Analysis ---")
print(f"Average Mana Value: {avg_mana_value:.2f}")
print(f"Total Creatures: {num_creatures}")
print(decklist_enriched[["name", "count", "cmc", "type_line"]])



--- Enhanced Deck Analysis ---
Average Mana Value: 1.68
Total Creatures: 14
                       name  count  cmc                              type_line
0            Bakersbane Duo      1  2.0            Creature — Squirrel Raccoon
1                Cache Grab      2  2.0                                Instant
2   Camellia, the Seedmiser      1  3.0  Legendary Creature — Squirrel Warlock
3       Cindering Cutthroat      1  3.0             Creature — Lizard Assassin
4            Daggerfang Duo      1  3.0                Creature — Rat Squirrel
5         Downwind Ambusher      1  4.0              Creature — Skunk Assassin
6        Druid of the Spade      1  3.0                Creature — Rabbit Druid
7                    Forest      9  0.0                    Basic Land — Forest
8             Glidedive Duo      1  5.0                  Creature — Bat Lizard
9      Hazardroot Herbalist      1  3.0                Creature — Rabbit Druid
10         Hazel's Nocturne      1  4.0               

In [17]:
SCRYFALL_CSV = "data/bloomburrow/cards.csv"
import requests
import os
def fetch_scryfall_data():
    if os.path.exists(SCRYFALL_CSV):
        print("Loading Scryfall data from existing file...")
        return pd.read_csv(SCRYFALL_CSV)
    
    print("Fetching Scryfall API data...")

    url = "https://api.scryfall.com/cards/search?q=set:blb"
    all_data = []  

    while url:
        response = requests.get(url)
        response_data = response.json()
        
        all_data.extend(response_data["data"])
        
        # Check if there are more pages
        url = response_data.get("next_page", None)  # Fetch next page if available

    # Convert full dataset to DataFrame and save
    df = pd.DataFrame(all_data)
    df.to_csv(SCRYFALL_CSV, index=False)
    
    return df

cards_df = fetch_scryfall_data()

Fetching Scryfall API data...
